In [17]:
import numpy as np
import pandas as pd

In [4]:
#     self.inputs = [.05, .1]
#     self.desired = [.01, .99]
#     self.learning_rate = 0.5
#     self.layers = [[Neuron(.05), Neuron(.1), Neuron(1)]]
#     self.layers += [[Neuron([e.g for e in self.layers[0]], np.array([.15, .2, .35])), 
#                      Neuron([e.g for e in self.layers[0]], np.array([.25, .3, .35])), 
#                      Neuron(1)]]
#     self.layers += [[Neuron([e.g for e in self.layers[1]], np.array([.4, .45, .6])), 
#                      Neuron([e.g for e in self.layers[1]], np.array([.5, .55, .6]))]]


In [5]:
sigmoid = lambda x: 1 / (1 + np.exp(-x))

In [6]:
class Neuron:
  def __init__(self, x=None, w=None, function=sigmoid):
    self.x = x
    self.u = None
    self.pd = None
    self.function = function
    
    if type(self.x) is float or type(self.x) is int:
      # Bias ou entrada
      self.w = None
      self.g = self.x
    elif type(self.x) is list:
      # Normal
      self.w = w if type(w) is np.ndarray else np.random.random(len(self.x))
      self.u = self.x @ self.w
      self.g = self.function(self.u)
    else:
      # Não-inicializado
      self.w = w if w is None or type(w) is np.ndarray else np.random.random(w)
      self.g = None
      
  def update(self, new_x):
      self.x = new_x
      if not self.is_bias():
        self.u = self.x @ self.w
        self.g = self.function(self.u)
      else:
        self.g = self.x
  
  is_bias = lambda self: True if self.w is None else False
  
  __repr__ = lambda self: '{} @ {} -> {}'.format(self.x, self.w, self.g)

In [45]:
class MLP:
  def __init__(self, topology):
    self.topology = topology
    
    self.init_neurons()   
    
  def init_neurons(self):
    # Camada de entrada
    self.layers = [[Neuron() for i in range(self.topology[0])]]
    self.layers[0] += [Neuron(1)] # Bias 
    
    # Camadas ocultas e de saída
    for i in range(1, len(self.topology)):
      self.layers += [[Neuron(w=len(self.layers[i - 1])) for j in range(self.topology[i])]]
      if i != len(self.topology) - 1: self.layers[i] += [Neuron(1)] # Bias

  def update_neurons(self): 
    # Camada de entrada
    [self.layers[0][i].update(self.inputs[i]) for i in range(len(self.inputs))]
    
    # Camadas ocultas e de saída
    for i in range(1, len(self.layers)):
      [n.update([e.g for e in self.layers[i - 1]]) for n in self.layers[i] if not n.is_bias()]
        
  def update_weights(self):
    # Camada de saída
    for i in range(len(self.layers[-1])):
      # Derivada parcial
      neuron = self.layers[-1][i]
      neuron.pd = - (self.desired[i] - neuron.g) * neuron.g * (1 - neuron.g)
      # Pesos
      for j in range(len(neuron.w)):
        neuron.w[j] -= self.learning_rate * neuron.pd * self.layers[-2][j].g
          
    # Camadas ocultas
    for i in range(len(self.layers) - 2, 0, -1):
      for j in range(len(self.layers[i]) - 1):
        # Derivada parcial
        neuron = self.layers[i][j]
#         print(neuron)
        descendents = sum([n.pd * n.w[j] for n in self.layers[i + 1] if not n.is_bias()])
        neuron.pd = descendents * neuron.g * (1 - neuron.g)
        # Pesos
        for k in range(len(neuron.w)):
          neuron.w[k] -= self.learning_rate * neuron.pd * neuron.x[k]
  
  def update_errors(self):
    out_layer = self.layers[-1]
    out_length = len(out_layer)
    
    self.errors = [(out_layer[i].g - self.desired[i]) ** 2 for i in range(out_length)]
    self.total_error = sum(self.errors) / 2
    
  def train(self, test_data, learning_rate=.1, precision=10**-6):
    self.learning_rate = learning_rate
    self.precision = precision
    self.epochs = 0
    
    # Erro inicial
    errors_sum = 0
    for index, sample in test_data.iterrows():
      self.inputs = sample[:self.topology[0]].tolist()
      self.desired = sample[self.topology[0]:].tolist()
      self.update_neurons()

      self.update_errors()
      errors_sum += self.total_error
    current_mse = errors_sum / test_data.shape[0]
      
    # Minimização dos erros  
    while True:
      last_mse = current_mse
      
      errors_sum = 0
      for index, sample in test_data.iterrows():
        self.inputs = sample[:self.topology[0]].tolist()
        self.desired = sample[self.topology[0]:].tolist()
        self.update_neurons()
        
        self.update_errors()
        errors_sum += self.total_error
        
        self.update_weights()

      self.update_neurons()
      self.epochs += 1
      
      current_mse = errors_sum / test_data.shape[0]
      print('{:4d} {:.9f} {:.9f} {:.9f}'.format(self.epochs, last_mse, current_mse, abs(current_mse -  last_mse)))
      if abs(current_mse -  last_mse) <= self.precision: break
        
  def test_individual(self, input):
    
  
  def __repr__(self):
    result = 'Camada 0 - Entrada\n{}\n\n'.format(self.layers[0])
    for i in range(1, len(self.layers) - 1): result += 'Camada {} - Escondida \n{}\n\n'.format(i, self.layers[i])
    result += 'Camada {} - Saída\n{}\n\n'.format(len(self.layers) - 1, self.layers[-1])
    
    return result

In [33]:
df = pd.read_csv('../../datasets/ressonancia.csv')
df = df.iloc[:3]

In [47]:
a = MLP([2, 3, 2, 2])
a.train(df)
a

   1 0.123484805 0.121924274 0.001560531
   2 0.121924274 0.117058261 0.004866013
   3 0.117058261 0.112082849 0.004975412
   4 0.112082849 0.107019440 0.005063410
   5 0.107019440 0.101892986 0.005126453
   6 0.101892986 0.096731629 0.005161358
   7 0.096731629 0.091566102 0.005165527
   8 0.091566102 0.086428935 0.005137167
   9 0.086428935 0.081353475 0.005075460
  10 0.081353475 0.076372794 0.004980681
  11 0.076372794 0.071518556 0.004854238
  12 0.071518556 0.066819936 0.004698620
  13 0.066819936 0.062302674 0.004517262
  14 0.062302674 0.057988338 0.004314336
  15 0.057988338 0.053893839 0.004094499
  16 0.053893839 0.050031225 0.003862614
  17 0.050031225 0.046407736 0.003623488
  18 0.046407736 0.043026089 0.003381647
  19 0.043026089 0.039884937 0.003141153
  20 0.039884937 0.036979446 0.002905491
  21 0.036979446 0.034301938 0.002677508
  22 0.034301938 0.031842531 0.002459407
  23 0.031842531 0.029589756 0.002252775
  24 0.029589756 0.027531109 0.002058646
  25 0.027531109

Camada 0 - Entrada
[0.6796 @ None -> 0.6796, 0.4117 @ None -> 0.4117, 1 @ None -> 1]

Camada 1 - Escondida 
[[0.6796, 0.4117, 1] @ [0.56029153 0.03714048 0.49353213] -> 0.7088069973818766, [0.6796, 0.4117, 1] @ [0.3328275  0.80731337 0.84477668] -> 0.8027129120083004, [0.6796, 0.4117, 1] @ [0.03684758 0.91394674 0.64014878] -> 0.7391320517763066, 1 @ None -> 1]

Camada 2 - Escondida 
[[0.7088069973818766, 0.8027129120083004, 0.7391320517763066, 1] @ [0.96903565 0.73587105 0.13214993 0.84394597] -> 0.90195823620468, [0.7088069973818766, 0.8027129120083004, 0.7391320517763066, 1] @ [0.09459539 0.76900395 0.83386625 0.78286773] -> 0.8892934514065401, 1 @ None -> 1]

Camada 3 - Saída
[[0.90195823620468, 0.8892934514065401, 1] @ [-0.32360978 -0.56342688  0.08211836] -> 0.32941856428830285, [0.90195823620468, 0.8892934514065401, 1] @ [ 0.62064715 -0.08168467  0.42077761] -> 0.7125771641854513]
